In [28]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
import torchvision.models as models 
from data_treatment import DataSet, DataAtts
from discriminator import *
from generator import *
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import glob
import os
from itertools import cycle
from numpy import genfromtxt
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.multioutput import MultiOutputRegressor
from torch.autograd import Variable
import time
import os

# Load Model


In [30]:
discriminator = DiscriminatorNet(192)
checkpoint = torch.load('models/discriminator.pt')
discriminator.load_state_dict(checkpoint['model_state_dict'])
discriminator.eval() 

generator = GeneratorNet(192)
checkpoint = torch.load('models/generatorpeptide.pt')
generator.load_state_dict(checkpoint['model_state_dict'])
generator.eval()

GeneratorNet(
  (hidden_0): Sequential(
    (0): Linear(in_features=100, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (hidden_1): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (hidden_2): Sequential(
    (0): Linear(in_features=512, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (out): Sequential(
    (0): Linear(in_features=1024, out_features=192, bias=True)
  )
)

In [31]:
# Noise
def noise(size):
    n = Variable(torch.randn(size, 100))
    if torch.cuda.is_available(): 
        return n.cuda() 
    return n

# Start with random seeds
newdata = generator(noise(30000)).detach().cpu().numpy()


# Set some parameter values

AAlist=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

HydrophobicityValues=[0.61, 1.07, 0.46, 0.47, 2.02, 0.07, 0.61, 2.22, 1.15, 1.53, 1.18, 0.06, 1.95, 0.0, 0.6, 0.05, 0.05, 1.32, 2.65, 1.88]
Alpha_CH_Chem_ShiftValues=[4.35, 4.65, 4.76, 4.29, 4.66, 3.97, 4.63, 3.95, 4.36, 4.17, 4.52, 4.75, 4.44, 4.37, 4.38, 4.5, 4.35, 3.95, 4.7, 4.6]
Conf_Par_of_Beta_StructureValues = [0.77, 0.65, 0.65, 0.55, 0.98, 0.65, 0.83, 0.98, 0.55, 0.83, 0.98, 0.55, 0.55, 0.72, 0.72, 0.55, 0.83, 0.98, 0.77, 0.83]
Conf_Par_of_Inner_HelixValues=[1.0, 0.06, 0.44, 0.73, 0.6, 0.35, 0.6, 0.73, 0.6, 1.0, 1.0, 0.35, 0.06, 0.44, 0.52, 0.35, 0.44, 0.82, 0.73, 0.44]
Conf_Par_of_Beta_TurnValues=[0.37, 0.84, 0.97, 0.53, 0.53, 0.97, 0.75, 0.37, 0.75, 0.53, 0.64, 0.97, 0.97, 0.64, 0.84, 0.84, 0.75, 0.37, 0.97, 0.84]
Residue_VolumeValues=[52.6, 68.3, 68.4, 84.7, 113.9, 36.3, 91.9, 102.0, 105.1, 102.0, 97.7, 75.7, 73.6, 89.7, 109.1, 54.9, 71.2, 85.1, 135.4, 116.2]
Steri_ParValues=[0.52, 0.62, 0.76, 0.68, 0.7, 0.0, 0.7, 1.02, 0.68, 0.98, 0.78, 0.76, 0.36, 0.68, 0.68, 0.53, 0.5, 0.76, 0.7, 0.7]
Norm_Freq_of_Beta_Turn_aValues=[0.66, 1.19, 1.46, 0.74, 0.6, 1.56, 0.95, 0.47, 1.01, 0.59, 0.6, 1.56, 1.52, 0.98, 0.95, 1.43, 0.96, 0.5, 0.96, 1.14]
Norm_Freq_of_Alpha_Helix_aValues=[1.42, 0.7, 1.01, 1.51, 1.13, 0.57, 1.0, 1.08, 1.16, 1.21, 1.45, 0.67, 0.57, 1.11, 0.98, 0.77, 0.83, 1.06, 1.08, 0.69]
Norm_Freq_of_Beta_SheetValues=[0.83, 1.19, 0.54, 0.37, 1.38, 0.75, 0.87, 1.6, 0.74, 1.3, 1.05, 0.89, 0.55, 1.1, 0.93, 0.75, 1.19, 1.7, 1.37, 1.47]
Beta_Strand_IndicesValues=[0.84, 1.27, 0.59, 0.57, 1.15, 0.94, 0.81, 1.29, 0.86, 1.1, 0.88, 0.66, 0.8, 1.02, 1.04, 1.05, 1.2, 1.56, 1.15, 1.39]
Alpha_Helix_IndicesValues=[1.29, 0.79, 1.1, 1.49, 1.13, 0.63, 1.33, 1.05, 1.33, 1.31, 1.54, 0.81, 0.63, 1.07, 1.0, 0.78, 0.77, 0.81, 1.18, 0.71]


# find the closest amino acid in parameter space using scaled L1 norm
def closestAA(i,k): 

    d1min = 10000.0;
    
    for j in range(0,20): # loop over 20 amino acids

        # distance function is a scaled L1 norm
        
        d1 = 0.0
        d1 = abs(HydrophobicityValues[j]-newdata[i,12*k])/(max(HydrophobicityValues)-min(HydrophobicityValues))
        d1 = d1+abs(Alpha_CH_Chem_ShiftValues[j]-newdata[i,12*k+1])/(max(Alpha_CH_Chem_ShiftValues)-min(Alpha_CH_Chem_ShiftValues))
        d1 = d1+abs(Conf_Par_of_Beta_StructureValues[j]-newdata[i,12*k+2])/(max(Conf_Par_of_Beta_StructureValues)-min(Conf_Par_of_Beta_StructureValues))
        d1 = d1+abs(Conf_Par_of_Inner_HelixValues[j]-newdata[i,12*k+3])/(max(Conf_Par_of_Inner_HelixValues)-min(Conf_Par_of_Inner_HelixValues))
        d1 = d1+abs(Conf_Par_of_Beta_TurnValues[j]-newdata[i,12*k+4])/(max(Conf_Par_of_Beta_TurnValues)-min(Conf_Par_of_Beta_TurnValues))
        d1 = d1+abs(Residue_VolumeValues[j]-newdata[i,12*k+5])/(max(Residue_VolumeValues)-min(Residue_VolumeValues))
        d1 = d1+abs(Steri_ParValues[j]-newdata[i,12*k+6])/(max(Steri_ParValues)-min(Steri_ParValues))
        d1 = d1+abs(Norm_Freq_of_Beta_Turn_aValues[j]-newdata[i,12*k+7])/(max(Norm_Freq_of_Beta_Turn_aValues)-min(Norm_Freq_of_Beta_Turn_aValues))
        d1 = d1+abs(Norm_Freq_of_Alpha_Helix_aValues[j]-newdata[i,12*k+8])/(max(Norm_Freq_of_Alpha_Helix_aValues)-min(Norm_Freq_of_Alpha_Helix_aValues))
        d1 = d1+abs(Norm_Freq_of_Beta_SheetValues[j]-newdata[i,12*k+9])/(max(Norm_Freq_of_Beta_SheetValues)-min(Norm_Freq_of_Beta_SheetValues))
        d1 = d1+abs(Beta_Strand_IndicesValues[j]-newdata[i,12*k+10])/(max(Beta_Strand_IndicesValues)-min(Beta_Strand_IndicesValues))
        d1 = d1+abs(Alpha_Helix_IndicesValues[j]-newdata[i,12*k+11])/(max(Alpha_Helix_IndicesValues)-min(Alpha_Helix_IndicesValues))
            
        if d1 < d1min:
            d1min = d1
            indmin = j
    
    return indmin

# convert a feature vector to its "closest" sequence
def feature2sequence(ind,featurevector):
    sequence = "                 "
    list1 = list(sequence)
    for k in range(0,16):
        indmin = closestAA(ind,k)
        list1[k] = AAlist[indmin]
    sequence = ''.join(list1)
    #print(sequence)
    return sequence





In [32]:
# normalize the feature data to correspond to a real AA sequence

for i in range(0,len(newdata[:,1])):  
        
    for k in range(0,16): # number of residues in peptide


        indmin = closestAA(i,k)
        
        # reassign descriptor values to correspond to a real sequence
        
        newdata[i,12*k] =   HydrophobicityValues[indmin]     
        newdata[i,12*k+1] = Alpha_CH_Chem_ShiftValues[indmin] 
        newdata[i,12*k+2] = Conf_Par_of_Beta_StructureValues[indmin]
        newdata[i,12*k+3] = Conf_Par_of_Inner_HelixValues[indmin]
        newdata[i,12*k+4] = Conf_Par_of_Beta_TurnValues[indmin]
        newdata[i,12*k+5] = Residue_VolumeValues[indmin]
        newdata[i,12*k+6] = Steri_ParValues[indmin]
        newdata[i,12*k+7] = Norm_Freq_of_Beta_Turn_aValues[indmin]
        newdata[i,12*k+8] = Norm_Freq_of_Alpha_Helix_aValues[indmin]
        newdata[i,12*k+9] = Norm_Freq_of_Beta_SheetValues[indmin]
        newdata[i,12*k+10] = Beta_Strand_IndicesValues[indmin]
        newdata[i,12*k+11] = Alpha_Helix_IndicesValues[indmin]
        
        
                                                                        

# Classification with DCNet

In [33]:
Xpep=np.array(newdata, dtype=np.float64)

In [34]:
# Pass test data
from torch.autograd import Variable
XpepT = torch.FloatTensor(Xpep)
Xpepy_hat_test = discriminator(XpepT)  
Xpepy_hat_test_class = np.where(Xpepy_hat_test.detach().numpy()<0.5, 0, 1)

In [35]:
Xpepy_hat_test_classContinuous=Xpepy_hat_test.detach().numpy()

In [36]:
len(Xpepy_hat_test_classContinuous)

30000

# Collect new Beta-hairpins 

In [37]:
for i in range(len(Xpep[:,1])):
    peptideID='peptide'+str(i)
    peptideSeq=feature2sequence(i,Xpep[i,:])
    Xpepi=np.array(Xpep[i,:], dtype=np.float64)
    XpepiT = torch.FloatTensor(Xpepi)
    Xpepiy_hat_test = discriminator(XpepiT)
    Xpepiy_hat_test_classContinuous=Xpepiy_hat_test.detach().numpy()
    if Xpepiy_hat_test_classContinuous<=.05:
        print(peptideID, peptideSeq, Xpepiy_hat_test_classContinuous[0])

peptide7 MKNDERDKFCFYTWDR  0.004854728
peptide61 LRKQKYQRHKCQGLEA  0.03215911
peptide86 YSHLDGVDKYLTLAKG  0.04311963
peptide257 FWFHVQWMNLASDHGF  0.016406426
peptide268 DKDMERFGFWYMHKRS  0.02257261
peptide352 LKHHYGQHFYICSQSG  0.0015155171
peptide545 VWNWKCCDIYTNNDAA  0.0032574341
peptide690 YLEWNSFTKLQGREHL  0.002379648
peptide695 AVYGSLEEMTHDVLHD  0.034756303
peptide698 FYEMFSVDFTRNSGKK  0.049185026
peptide793 SKMDGGTLYGKRVYQW  0.042631686
peptide1110 IWEIKVTHLYWCYKEQ  0.020050237
peptide1117 VKDIHSQMDCCTGCEI  0.04887211
peptide1121 IMHMKYTPAADSRLES  0.042132966
peptide1258 HVRMHWVDMIHLFMDE  0.04295319
peptide1264 VKDHTDKRQIYTLTGI  0.0194071
peptide1354 HFHKHMWMDFDFFDMH  0.038590025
peptide1634 LLRHTRVKHMKSKSRL  0.022984387
peptide1715 RDHHHRIDLYCVSMSD  0.022991821
peptide1749 YRKITGAHCKQQFAEV  0.023596838
peptide1995 AGFWQYAYYDLDMKDT  0.041918524
peptide1998 HEHFGYMVHTINMQGC  0.020935422
peptide2037 ILRMDGECDRRSSLAL  0.049045462
peptide2076 EHIPVHIQTKKHYDNA  0.026478512
peptide2159 

peptide14254 LQLFLADFEHLKCSLH  0.016573597
peptide14294 KLNDDLLGLYKHTMQS  0.024784068
peptide14367 MMWSTVDMEKHKHFFH  0.0365486
peptide14397 SMHGHTHTKYKWMHKR  0.007842725
peptide14432 ETHCHLDKQFDLNKSH  0.017109485
peptide14470 FYFHKCQHFYHHLIEE  0.032630112
peptide14488 WCHMSIMTFKMELMSV  0.018703818
peptide14561 QYDNHKKHIYTLGAGL  0.005346947
peptide14596 VFHHVYDLFHHSSHAA  0.005676875
peptide14757 FWHLFGQHLLHGVDMS  0.0012786594
peptide14928 FIDFDYDHHHSDFMHD  0.03503507
peptide14954 VKTHSGMDYKCNDIQF  0.038011216
peptide15077 LYQDCGAENKFCSDHC  0.0066541326
peptide15212 HCRALVENWLTHVDMK  0.040023103
peptide15218 ILKIMNQDIYTCRELV  0.045320578
peptide15234 GAHPDQYKRTKKYMYN  0.027761977
peptide15256 LLTTVGLNYYLNTDFV  0.041150905
peptide15376 VACHSWTHILCYGLEI  0.038006864
peptide15407 KCHQMGHGLCWYERRS  0.04191096
peptide15441 RYHFDTCDYYTCGAEI  0.028529683
peptide15656 MYMHDVMDVYWPEKGW  0.009309898
peptide15672 LRELRTDHITHKYITK  0.040453978
peptide15697 LKLLIGPDITTSGEEG  0.0020231004
peptide15719

peptide27675 ALDHTVEHKKFIEFEA  0.023635376
peptide27705 IHHKTTATEDHGKLKF  0.04479449
peptide27758 ATNNSIFMFGVHTAGD  0.015510687
peptide27831 YHHLKLQDIVHHFLEE  0.033062875
peptide27891 ASGNTRSLVGQVGYQK  0.017008103
peptide28160 IRKLHSIRKLPTRMLA  0.0131290285
peptide28178 DIHDTQNEKQHKPKVI  0.0388448
peptide28223 NMMWVCMKPLMDYTNY  0.039388765
peptide28283 VEHYLPERTHTSGADY  0.030106045
peptide28360 VGDILRQHIPVRSAEV  0.03238176
peptide28424 LLDLHYEMMYTLQLDA  0.011670422
peptide28653 AVHNQRKQLCRQLFSK  0.02629785
peptide28765 KHTEKDQDKIRYDMKS  0.01990111
peptide28813 LVFGVVDLFDWKDFFI  0.010894843
peptide29032 MVTCEMAHYYIWTYDD  0.032913815
peptide29268 IKFHLGTDICSTNAES  0.0075234952
peptide29360 WKQHNPSDMFHSTMWD  0.014923431
peptide29391 HLYSHFSQHMSHFFDD  0.045536168
peptide29393 HMYHICHRCYLGSQDV  0.041861914
peptide29402 LKSPQAYQPQGQAHKT  0.044796336
peptide29507 VREPHRSFVGQCDYTH  0.04404401
peptide29618 QCGHHGTHKDTNTSQL  0.021536987
peptide29676 FCDGLVDMFEMEKYKY  0.0026942485
peptide29680 IL

In [38]:
for i in range(len(Xpep[:,1])):
    peptideID='peptide'+str(i)
    peptideSeq=feature2sequence(i,Xpep[i,:])
    Xpepi=np.array(Xpep[i,:], dtype=np.float64)
    XpepiT = torch.FloatTensor(Xpepi)
    Xpepiy_hat_test = discriminator(XpepiT)
    Xpepiy_hat_test_classContinuous=Xpepiy_hat_test.detach().numpy()
    if Xpepiy_hat_test_classContinuous>=.95:
        print(peptideID, peptideSeq, Xpepiy_hat_test_classContinuous[0])

peptide0 TKQVEVNGSLYRGTSS  0.9680029
peptide2 VYVEGSVNGLRFRCHG  0.9986395
peptide4 STHLMMHDGWMVFRYM  0.97745186
peptide6 LKWFLTGFDGPFTLLV  0.9719111
peptide10 QFICFRCMGKQYNIRV  0.96341425
peptide17 DDVRIELCGKKFIGCV  0.9842241
peptide20 FNHIMGGWGSQFHVLF  0.9941924
peptide22 DQIFIMFCGHKMINYI  0.98729336
peptide23 VVLVRTSWDSTHMMYN  0.99493295
peptide31 VMFKGNFNDVQFKFHS  0.9536974
peptide32 SKLLILKGSQQFGTAH  0.9751418
peptide36 IKFLVYSGRRIFIRRS  0.9895253
peptide38 TLMFYCQHGQNWLMYW  0.9790982
peptide44 MHFILQREGRLLYVGL  0.99628395
peptide45 LRMEGAVDGHRFVIEG  0.9985436
peptide53 QMMHGHVNGHEFHFKG  0.99345535
peptide57 LYMEGTVNGKLFKCVG  0.99947935
peptide58 HLFAKYSGDRLWLKGS  0.96104354
peptide62 LIWMWEDMGGYFKHRK  0.99116766
peptide63 YSYVVELDGFMFSSMD  0.96989113
peptide75 WYFVWDYDGRKMMTMG  0.9886961
peptide76 DQIIIEIAGHKAIGSV  0.9925666
peptide79 MFFFLMYHGYFWTNRF  0.9720305
peptide82 AGQIYYTGRGVVGVGV  0.9739086
peptide88 SVFLDMWEGHSVLWYS  0.9616104
peptide89 TGALMTTGRGVMVSVH  0.96145785
pepti

peptide761 VRCLTTVPGIQFLSTS  0.9869871
peptide771 IQFYMHKDGFFFKDDV  0.9860504
peptide779 DQIIIEFCGRKAIGTV  0.99910694
peptide780 HYQRRLFGGIPFLFVL  0.98749024
peptide785 SAHNDICKDGDCYHYE  0.9734588
peptide794 LLVQVFSGDQTYCLWN  0.99187005
peptide797 TYVRLQANGKQMVGMD  0.9811265
peptide801 QVGLTQVKGQRFMVVK  0.9924468
peptide810 HFAFHMWDGDKFWIVF  0.9879133
peptide812 DQLPIEICGHKVIGCV  0.9958781
peptide813 SFFLMFRDDRLMYQAD  0.9821975
peptide817 QQIIIEISGHKAIGTV  0.99882144
peptide820 TLKNDCDFDGNCFYYD  0.99361247
peptide831 ALMLQKYHGKKFILTR  0.97429556
peptide832 KLLEGAQNGQKFAIKG  0.99783957
peptide839 DHIPVHICGHKAIGSI  0.9770682
peptide843 SYLPFSIDGTYVRWYT  0.9948672
peptide844 QLQFAHNGDSFYTRTT  0.9727381
peptide850 HCKLTSHNGGLLLYYE  0.9940394
peptide853 KIIPVEYSGQQVLGTV  0.9976392
peptide860 DLLFLKCDGHFIAKFI  0.97731745
peptide862 VFQDDIRQGDKYLIFE  0.9555873
peptide863 LWMEGTVNGHHFLVKG  0.99896216
peptide868 YTILMRRSRKVFLFII  0.98181146
peptide873 IIGFAHLDGEHYWFSL  0.979932
peptide875 SMFLE

peptide1647 RQVVHAFHGRHIFEWQ  0.9735309
peptide1648 NIMITIQDGFLMLSDD  0.9803028
peptide1654 LLIAQISEGEVLILRD  0.95038474
peptide1655 QQVIATEGDNFQVTST  0.9743682
peptide1656 LQVQILSHGQQLLTLS  0.9800102
peptide1657 SWNVHRKSGRKHFNRH  0.958071
peptide1660 QFVFMHFHSIRFFMWN  0.9561147
peptide1662 VRVQIKLDGKFLHSHN  0.99662113
peptide1664 ALVCVMYAGELVWRWR  0.98967326
peptide1672 VHSLTNHDGYMFLQYK  0.9592158
peptide1674 SAAQEQCHNGQCLIYH  0.97437876
peptide1676 DQIPIEICGHKAIGTV  0.9939427
peptide1677 KDLQVTQSGGQFRLSH  0.97356105
peptide1680 DHIPFMFCGHKVIGCF  0.98832035
peptide1684 DQIIIEFCGHKVFGTI  0.99814653
peptide1696 RLYRLVQGDDYQLAAD  0.9519166
peptide1697 STRYMENGGNYYMRYF  0.99314946
peptide1701 NVIIQLKGDIIFIDSE  0.9744645
peptide1702 AALYWKNDGTWFPLWQ  0.9918174
peptide1704 AFFFLWCMGRWFVAAF  0.99769443
peptide1705 ARIQYTYDGKAVFLDG  0.9670132
peptide1707 FRIMFFDDGELYIKFN  0.98449636
peptide1708 HYEILQDGDHMWVLYK  0.96462446
peptide1711 DKFYILLNGNRFFRCI  0.99316543
peptide1713 ILWTFENEGQTVSFTT 

peptide2437 MIEFQERDGKLTMFFL  0.9543617
peptide2438 VKIHTDYDGRRFSLSG  0.99888176
peptide2440 LHLEGTVSGHQFTISG  0.9989465
peptide2441 LYMEGQVNGHKFIIEG  0.9993838
peptide2442 MIVLFDFGTLLYLMFR  0.95192933
peptide2450 SVHTQVRGDTLLLAGD  0.9791403
peptide2451 LYRHGFVNGRLFSTDR  0.9689034
peptide2453 VFSMAWTGRRICAVHF  0.98118836
peptide2454 MFEIDLWDGMMWIFTK  0.9760082
peptide2458 CATNDITLDGNCYFYH  0.9954478
peptide2460 TRLCLSYPSGRILGFM  0.9916381
peptide2468 TTWYRTLSGHYFMYTW  0.9962381
peptide2470 RVLILLDGGVLFQEGR  0.96943384
peptide2472 LHMEGHVNGHEFVIEG  0.9970271
peptide2474 QHMEGTVNGHEFEIEG  0.99319893
peptide2481 DQIIIKFKGHKAISCV  0.9958307
peptide2484 VKFDGDVDGHKFNIHG  0.99739945
peptide2486 DQFFIMLAGQHYFGCI  0.9767371
peptide2489 TLRHLLFSGRWMSCYV  0.99594575
peptide2490 KLLKYEFDGKNIHFRV  0.9680329
peptide2492 WTLFVMFAGRFFNMGR  0.98726195
peptide2496 LVQLQKDGDNFYIYFN  0.9825853
peptide2497 FLVVSIQDNSFYARKS  0.97223175
peptide2501 VLLREQRHGYKTVKFC  0.98093814
peptide2504 TTKLVSYNGKRVHVCW  

peptide3234 YVIMIDHPQNYLVGLE  0.9654322
peptide3235 KTLDVQINGTCVHSRI  0.95446175
peptide3240 YFLNVAVDGQTQHIEI  0.9712072
peptide3241 LKMEGTVSGHEFRVSG  0.9973362
peptide3244 GWFVMMEHSYLLVIRA  0.97934365
peptide3246 HDFPFSMDNGLFFHYH  0.9663906
peptide3248 LFFSGYHERGSYVFLV  0.9692824
peptide3250 VVIYLIRHDDVVWKYL  0.9956357
peptide3251 VHLDGHVDGMHFSQSG  0.99149346
peptide3252 FVHTVVYGTRQLLLGK  0.9820956
peptide3258 DQIRIEICGHKAIGTV  0.9950937
peptide3260 LHMEGTVNGHEFEIES  0.9895794
peptide3265 LPKIIGGWGGIIHVHL  0.99970347
peptide3266 TEFDTHVNGHRFSISS  0.97585344
peptide3273 RRHEGAVNGHPYTIEG  0.9965922
peptide3279 IFTHVQKDGMHHLTMM  0.96828103
peptide3283 DHIPIEFKGHKAISTV  0.97664964
peptide3287 HHLNVAINGHKAIGTI  0.9790997
peptide3290 MGKNVCHKTGFVFVHN  0.98192376
peptide3292 DHIFIHFHGHKMIGYF  0.9839087
peptide3294 KMLRHFAGGHFFLAHY  0.9631579
peptide3295 SYFIAFDNNQFKRVFM  0.98879445
peptide3296 VELDGDVNGDRFSVSG  0.9916653
peptide3304 GTFSFKWDSGKLKVYK  0.9707342
peptide3305 MVIYILKKDQIVKWWL  0

...

peptide26515 GVKHINYHGFNFHMCF  0.95112157
peptide26517 YCYRVQYDGQQYSSVG  0.99746406
peptide26523 AVQISQKGGDHLFYRY  0.96983373
peptide26527 VHVRVMSHGKQIGDLQ  0.9583228
peptide26542 GSRTITVGDGQLKVVI  0.99168104
peptide26547 LKLDGAVNGHPFDIKG  0.99523675
peptide26555 FIFTVYRGHHVQIQHD  0.9663636
peptide26558 DQIIIEICGHKAIGCV  0.99818546
peptide26559 KFMMGWYGDWMCIMFE  0.97564834
peptide26564 VLFDSTIDGHTFSLDQ  0.9641044
peptide26565 VMMFFHYMNMFFYMIM  0.950739
peptide26566 DLLVVVDGKQRFALCK  0.95532537
peptide26568 YFFFCHFEGFMQMLRM  0.96464723
peptide26570 DQIIIKQKSNKAFSTL  0.97115356
peptide26574 VKTDTSTDGQQYNQSG  0.95147437
peptide26575 SCWFIHYHGKQFTMCH  0.96771497
peptide26577 ALFCVHYNGHWFKKRV  0.99579453
peptide26579 VAWRFEAKEGPWHICV  0.9602639
peptide26583 CSELHEYSGQRIFMQT  0.9556898
peptide26591 VLQDGHVDGHKISVSG  0.99794835
peptide26593 VILIMQRHNKICYHID  0.9726855
peptide26602 VRDLLRTGSRQHHGVF  0.9664243
peptide26603 EIKVKQSGDRMVTWLL  0.95130956
peptide26604 DQIPIEFCGHKVIGCV  0.99768376
p

peptide27342 SHIVFTRDGVLRYISQ  0.9948289
peptide27345 LPMFLGGLGGLFDVQL  0.98663414
peptide27354 VEMTLNSGGRYYSQWY  0.97332394
peptide27363 IKLDGRVNGHKFLVEG  0.998168
peptide27366 FIFPSEFDGRLYKRWL  0.9804875
peptide27369 QLQIEQNGGKLRVVQI  0.99318415
peptide27375 LHLEGDVNGHKFIITG  0.99894387
peptide27376 DTWYMEFHGKMVIFNF  0.9640303
peptide27385 FWYFTYTDGKKVMFTH  0.99199414
peptide27389 DIWIQTQEDQYKYLAQ  0.9604748
peptide27392 VVMTTYHAGTRYVQQT  0.9827771
peptide27393 AITQHFYDGKLWLMDL  0.9695319
peptide27399 NIFLTYQGQRVSMRGQ  0.956436
peptide27401 YVFKFEDDSHFYDLFL  0.9632597
peptide27403 HMMESHFNGHEFEIES  0.95439553
peptide27405 IVWEGKFRGDHVAVSV  0.97268605
peptide27408 VVISTTSMRETVMLFN  0.98602515
peptide27409 TEMTWYASGDFLFLYY  0.976661
peptide27411 DLIFIEFCGHKAFGCV  0.9956702
peptide27412 LLLVVEWGSRHLPHMC  0.9872766
peptide27416 FLFGVVDGDQFFLFMN  0.99030674
peptide27417 DQILVEICGRKAIGSV  0.9967403
peptide27419 DQIIIEICGTKAIGTV  0.99679154
peptide27421 FILLRFNDGQRLGATC  0.9712295
peptide27

peptide28094 AIFTSHWKGMDFWIVR  0.9629184
peptide28095 IMLDGHVNSHKFVIKG  0.99718696
peptide28098 TLTQMTYKGMHTFHLQ  0.9627503
peptide28103 FRVLYAFDGHPLHMAH  0.9688099
peptide28105 TFIFEYSDDLLLFQLM  0.9500684
peptide28111 DQILIEICGHKAIGSV  0.99474734
peptide28114 DLLIKTKGDVMFILVS  0.9862483
peptide28115 YTYYVTRHGELMRSVC  0.99081236
peptide28133 NLFNICFCGGDFLTWD  0.97560453
peptide28136 DQIIIEICGHKAIGTV  0.9979874
peptide28138 VRVQLHLQGMKVIAMK  0.9958092
peptide28142 DQIPIEICGHKVIGTV  0.99732614
peptide28143 VKTGGQQGGKRVSVSG  0.99691087
peptide28147 DWYYMTFGGESYVQYT  0.97293067
peptide28158 KHMEGHVNGHHFFIDG  0.9921158
peptide28167 VKWYIYSGRNQFMMSW  0.98839134
peptide28170 IYHNGFINGDYFLVEE  0.9785962
peptide28174 KMLITFWGDKMFLLWD  0.978729
peptide28185 QFKWKLHDGWKTYQFL  0.97384465
peptide28189 VVKRYERDGRPIEVTV  0.99793905
peptide28191 QHLDGAVNGHKFQIKG  0.9904794
peptide28193 YVLMIQRHGHVVQKTF  0.99531424
peptide28207 VRLARDKEGSMVLSFH  0.96854365
peptide28220 VFLFDKQHGLWTFLFN  0.9838464
pepti

peptide28955 LHMEGNVNGHHFVIDG  0.99820757
peptide28961 TLRVVYVGQHQRIAGK  0.9831202
peptide28962 GFMMYCYDNHYYRWLF  0.9704982
peptide28971 VRLHGTHNGKQIKTIT  0.99844235
peptide28972 IHMEGQVNGREFAIQG  0.9994259
peptide28978 GMYSTYYGDWWCFRRM  0.97134596
peptide28979 FFKFSFFDGRMYLKFD  0.99692804
peptide28983 QCKFAGGWGGCYHVKQ  0.9985668
peptide28986 LWMEGAFNGHKFAIEG  0.9952289
peptide28987 YMMMTYWDGMMYVHWF  0.9745685
peptide28993 VIINISMKDSVVSRYV  0.98155963
peptide28999 QIIISCKGDRLFVHTK  0.98432136
peptide29003 STVTFMNHGKKIKIYT  0.9941454
peptide29007 DKLIIHMEGCRAIGTV  0.98287797
peptide29008 DQIIIEIAGHKAIGTV  0.99509346
peptide29017 QHIRIMLDGKKAWSHI  0.986968
peptide29019 SGLYLKSSGWWFFYDH  0.99098283
peptide29020 RLVILDRGSVRFQSAR  0.95474064
peptide29031 FAWTGVHGDMKFLWTM  0.9655762
peptide29035 DQIIIEFCGHKAIGSF  0.99194545
peptide29037 MYMFHWKDGEWYFLHF  0.97569364
peptide29039 LHLDGHIDGHNYQQAS  0.9788503
peptide29044 FIEMHIKDGWIYIDWN  0.98562616
peptide29047 VTRRFQIDGRTVKVAV  0.9974253
pept

peptide29707 MLQLMEFNGKHLMMYA  0.96812046
peptide29709 DQILIEICGRKAIGCV  0.9987205
peptide29715 AVLFEYNSNYLRAFTQ  0.9634573
peptide29717 VIFMVRKGKDICITRI  0.9650925
peptide29719 KVQVDTFKGMPFLRFD  0.97687006
peptide29722 GIHFALNDGMKWFMTK  0.9923213
peptide29723 LALFWYHNDHYKHIGW  0.9705995
peptide29725 FHLQFWSRGGHYWQWY  0.9950098
peptide29738 TWQHMYRDGMQQLTFT  0.9757734
peptide29741 KVVARFYSGLPFLRFD  0.9798251
peptide29767 DHIYIMFCGHKAIGNF  0.9861783
peptide29768 FMMMTHHASGTQFIFW  0.9540325
peptide29772 LRLEGAFNGKPYAIAS  0.99832314
peptide29780 WKLFVLEGSLHILFCY  0.96791905
peptide29785 HHIPVEFSGHKAIGNV  0.9855642
peptide29795 VQVCVKQQNLPVEHWR  0.9528203
peptide29799 DAIIIHIAGHRAINTV  0.9703447
peptide29800 DQILIEICGHKFFGCV  0.99287343
peptide29802 DQILIEFCGRKAIGTV  0.9985856
peptide29810 NEIIIQRSSTRAINSL  0.9711481
peptide29811 HMTVMDRNGFHMLMFD  0.9660614
peptide29814 RLVLSYFDGIAYIMRN  0.97345984
peptide29819 RVIILRRGHVIFAAMQ  0.9583779
peptide29820 ITWLYKMDGHFYHVRF  0.9948048
peptide298